In [ ]:
#Debug
#!pip install git+https://github.com/huggingface/accelerate
#!pip install --upgrade transformers

In [ ]:
!pip install -q transformers==4.34.0 datasets==2.14.5 accelerate==0.23.0 evaluate==0.4.1 peft==0.5.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 59.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 106.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 87.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 2.3 MB/s eta 0:00:00


In [ ]:
import numpy as np
import transformers
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

print(transformers.__version__)

4.34.0


In [ ]:
#Setup
import torch
def id_labeling(num_opts):
  option_dict = {2: ['a', 'b'],
                 3: ['a', 'b', 'c','ab', 'ac', 'bc','abc'],
                 4: ['a', 'b', 'c', 'd', 'ab', 'ac', 'ad', 'bc', 'bd', 'cd','abc', 'abd', 'acd', 'bcd','abcd'],
                 5: ['a', 'b', 'c', 'd', 'e', 'ab', 'ac', 'ad', 'ae', 'bc', 'bd', 'be', 'cd', 'ce', 'de', 'abc', 'abd', 'abe', 'acd',
                     'ace', 'ade', 'bcd', 'bce', 'bde', 'cde', 'abcd', 'abce', 'abde', 'acde', 'bcde','abcde']}
  if num_opts in [2,3,4,5]:
    label_list = option_dict[num_opts]
    id2label = {}
    label2id = {}
    for idx, label in enumerate(label_list):
      id2label[idx] = label
      label2id[label] = idx
    num_labels = len(id2label)
    return id2label, label2id, num_labels

# model_checkpoint = "dmis-lab/biobert-base-cased-v1.2"
# model_checkpoint = "dmis-lab/biobert-v1.1"
# model_checkpoint = "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext"
# model_checkpoint = "camembert-base"
# model_checkpoint = "xlm-roberta-base"
model_checkpoint = "allenai/scibert_scivocab_uncased"

id2label, label2id, num_labels = id_labeling(5)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint,
                                          num_labels=num_labels,
                                          label2id=label2id,
                                          id2label=id2label,
                                          use_fast=True)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint,
                                                           num_labels=num_labels)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at allenai/scibert_scivocab_uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#modified_MCQA (MedMCQA&FrenchMedMCQA)
!gdown 1e5FHk1tmAy_VmyfgTi42lnzvHUM2Zmo0
!unzip modified_MCQA.zip

Archive:  modified_MCQA.zip
  inflating: MCQA_dev.json           
  inflating: MCQA_train.json         


In [ ]:
from datasets import load_dataset
from datasets import DatasetDict

data_dir = '.'
raw_dataset = {
    'train': load_dataset('json', data_files=f'{data_dir}/MCQA_train.json')['train'],
    'valid': load_dataset('json', data_files=f'{data_dir}/MCQA_dev.json')['train']
}
raw_dataset = DatasetDict(raw_dataset)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
def preprocess_function(examples, max_seq_length, tokenizer):
  sentences = []
  labels = []
  for example in zip(examples['bert_text'], examples['label']):
      sentences.append(example[0])
      labels.append(example[1])
  model_inputs = tokenizer(sentences,
                           padding="max_length",
                           max_length=model.config.max_position_embeddings,
                           truncation=True)
  model_inputs["labels"] = torch.tensor(labels)
  return model_inputs

In [ ]:
from functools import partial
processed_dataset = raw_dataset.map(partial(preprocess_function,
                                            max_seq_length=model.config.max_position_embeddings,
                                            tokenizer=tokenizer),
                                    batched=True,
                                    load_from_cache_file=False,
                                    remove_columns= ['id', 'question', 'answer_a', 'answer_b', 'answer_c', 'answer_d', 'answer_e', 'label', 'context', 'bert_text'],
                                    desc="Running tokenizer on dataset",)

Running tokenizer on dataset:   0%|          | 0/185615 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/4495 [00:00<?, ? examples/s]

In [ ]:
#Hyper parameters
model_name = model_checkpoint.split("/")[-1]
task = "MedMCQA&FrenchMCQA"
#batch_size = 4
#batch_size = 6
batch_size = 16
#batch_size = 24
#batch_size = 32
EPOCHS = 3
metric_name = "accuracy"

#TrainingArguments
args = TrainingArguments(f"{model_name}-finetuned-{task}-v{7}",
                         evaluation_strategy = "steps",
                         save_strategy = "steps",
                         learning_rate=2e-5,
                         per_device_train_batch_size=batch_size,
                         per_device_eval_batch_size=batch_size,
                         num_train_epochs=EPOCHS,
                         save_steps = 5000,
                         eval_steps = 5000,
                         save_total_limit = EPOCHS,
                         weight_decay = 0.01,
                         load_best_model_at_end=True,
                         metric_for_best_model=metric_name,
                         push_to_hub=False,)

In [ ]:
#Metric
import numpy as np
import evaluate
from transformers import EvalPrediction

metric = evaluate.load("accuracy")
def compute_metrics(eval_pred: EvalPrediction):
  predictions, labels = eval_pred
  predictions = np.argmax(predictions, axis=1)
  result = metric.compute(predictions=predictions, references=labels)
  return result

In [ ]:
#Trainer
trainer = Trainer(model,
                  args,
                  train_dataset=processed_dataset['train'],
                  eval_dataset=processed_dataset['valid'],
                  tokenizer=tokenizer,
                  compute_metrics=compute_metrics,)

In [ ]:
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Accuracy
5000,0.801400,1.134060,0.519021
10000,0.726000,1.078486,0.547720
15000,0.661400,1.031431,0.546162
20000,0.668100,1.023039,0.556618
25000,0.625000,1.011556,0.566407
30000,0.602800,1.006554,0.567519


TrainOutput(global_step=30936, training_loss=0.7125367706432989, metrics={'train_runtime': 19295.9332, 'train_samples_per_second': 9.619, 'train_steps_per_second': 1.603, 'total_flos': 4.885007477779968e+16, 'train_loss': 0.7125367706432989, 'epoch': 1.0})